In [75]:
from arcgis.features import FeatureLayer

In [76]:
import datetime

In [77]:
import csv

In [78]:
import os

In [79]:
import pandas as pd

In [80]:
from geopandas.tools import sjoin

In [81]:
import geopandas as gpd

In [82]:
from shapely.geometry import Point

In [83]:
import pyproj

In [84]:
from pyproj import Transformer

In [85]:
camada_lotes = "https://sig.niteroi.rj.gov.br/server/rest/services/Hosted/NPG_SMF_SEREC_A_LOTES_EDIFIC/FeatureServer/30"

In [86]:
limite_bairros= r'T:\RECAD\BACKUPlotes\shp\limite_bairros_niteroi_wgs.shp'

In [87]:
bairros_gdf = gpd.read_file(limite_bairros, encoding='ISO-8859-1')

In [88]:
transformer = Transformer.from_crs("epsg:31983", "epsg:4326", always_xy=True)

In [89]:
def convert_utm_to_gcs(x, y):
    try:
        lon, lat = transformer.transform(float(x), float(y))
        return lon, lat
    except (ValueError, TypeError):
        return None, None

In [90]:
query = f"last_edited_date >= TIMESTAMP '{datetime.date.today() - datetime.timedelta(days=0)}'"

In [91]:
campos = ["tx_insct", "tx_loteabrev", "tx_nroproc", "tx_observ", "tx_logrado", "si_nroport", "tx_complem", "tx_quadraloc", "tx_loteloc", "tx_loteame", "tx_bairro", "db_v0", "tx_arealote", "tx_a_constr", "tx_nmfrente", "tx_testadap", "tx_nmprop", "dt_datareg", "si_livrode", "tx_rgi", "db_coordx", "db_coordy", "tx_streetview", "tx_faststreet", "tx_tipo_lo", "SHAPE__Length", "SHAPE__Area", "created_user", "created_date", "last_edited_user", "last_edited_date"]

In [92]:
caminho_csv = r"T:\RECAD\BACKUPlotes\BackupLOTESSMF.csv"

In [93]:
tx_insct_index = campos.index('tx_insct')

In [94]:
last_edited_date_index = campos.index('last_edited_date')

In [95]:
entradas_existentes = set()

In [96]:
contagem_datas = {}

In [97]:
with open(caminho_csv, 'r', newline='', encoding='ISO-8859-1') as file:
    reader = csv.reader(file, delimiter='$')
    next(reader)
    for row in reader:
        if len(row) > max(tx_insct_index, last_edited_date_index):
            chave = (row[tx_insct_index], row[last_edited_date_index])
            entradas_existentes.add(chave)
            data_edicao = row[last_edited_date_index]
            if data_edicao in contagem_datas:
                contagem_datas[data_edicao] += 1
            else:
                contagem_datas[data_edicao] = 1

In [98]:
with open(caminho_csv, 'a', newline='', encoding='ISO-8859-1') as file:
    escritor = csv.writer(file, delimiter='$')
    with arcpy.da.SearchCursor(camada_lotes, campos, where_clause=query) as cursor:
        for row in cursor:
            x_utm, y_utm = row[campos.index('db_coordx')], row[campos.index('db_coordy')]
            lon, lat = convert_utm_to_gcs(x_utm, y_utm)
            if lon is not None and lat is not None:
                point = gpd.GeoDataFrame([{'geometry': Point(lon, lat)}], crs='EPSG:4326')
                point_in_bairro = sjoin(point, bairros_gdf, how="left", predicate='within')
                bairro_name = point_in_bairro.iloc[0]['tx_nome'] if not point_in_bairro.empty else None
                row_modificado = [str(campo).strip() for campo in row]
                row_modificado[campos.index('db_coordx')] = lon
                row_modificado[campos.index('db_coordy')] = lat
                row_modificado[campos.index('tx_bairro')] = bairro_name
                chave_entrada = (row[tx_insct_index], row[last_edited_date_index])
                data_edicao = row[last_edited_date_index]
                if chave_entrada not in entradas_existentes and contagem_datas.get(data_edicao, 0) <= 10000:
                    escritor.writerow(row_modificado)
                    entradas_existentes.add(chave_entrada)
                    contagem_datas[data_edicao] = contagem_datas.get(data_edicao, 0) + 1